# Lab 17: Graphs: Breadth-First Searching

## <font color=DarkRed>Your Exercise: All Pairs Shortest Path</font>

Using breadth first search write an algorithm that can determine the shortest path from each vertex to every other vertex. This is called the all pairs shortest path problem.

## <font color=green>Your Solution</font>

*Use a variety of code, Markdown (text) cells below to create your solution. Nice outputs would be timing results, and even plots. You will be graded not only on correctness, but the clarity of your code, descriptive text and other output. Keep it succinct!*

In [207]:
from random import random
import sys

In [208]:
class Queue:
    def __init__(self):
        self.items = []
        
    def is_empty(self):
        return self.items == []
    
    def enqueue(self, item):
        self.items.insert(0, item) # this insert is actually a O(n)
        
    def dequeue(self):
        return self.items.pop()
    
    def size(self):
        return len(self.items)

In [209]:
class Vertex:
    def __init__(self, key):
        self.id = key
        self.connected_to = {}  # "adjacency list", represented a dict ;the key is vertex,value is weight
        self.color = "white"
        self.dist = sys.maxsize
        self.pred = None
        self.disc = 0
        self.fin = 0
        
    def __str__(self):
        return f"<Vertex({self.id}): connected to: {[x.id for x in self.connected_to]}"
    
    def __repr__(self):
        return f"Vertex({self.id!r})"
    
    def add_neighbor(self, nbr, weight=0):
        self.connected_to[nbr] = weight
        
    def set_color(self, color):
        self.color = color
        
    def set_distance(self, d):
        self.dist = d
        
    def set_pred(self, p):
        self.pred = p
        
    def set_finish(self, ftime):
        self.fin = ftime
        
    def get_finish(self):
        return self.fin
    
    def get_discovery(self):
        return self.disc
    
    def get_pred(self):
        return self.pred
    
    def get_distance(self):
        return self.dist
    
    def get_color(self):
        return self.color
    
    def get_connections(self):
        # returns the neighbors to this Vertex as a list of their Vertex objects
        return self.connected_to.keys()
    
    def get_weight(self, nbr):
        return self.connected_to[nbr]
    
    def get_id(self):
        return self.id

In [210]:
class Graph:
    def __init__(self):
        self.vertices = {} #key is the id of vertex，value is vertex
        self.num_vertices = 0
        
    def __contains__(self, key):
        return key in self.vertices
    
    def __iter__(self):
        return iter(self.vertices.values())
    
    def add_vertex(self, key):
        # What to do if this key already exists here?
        if key not in self.vertices:
            new_vertex = Vertex(key)
            self.vertices[key] = new_vertex
            self.num_vertices += 1
            
    def get_vertex(self, key):
        if key in self.vertices:
            return self.vertices[key]
        else:
            return None
        
        # Alternative one-liner
        # return self.vertices.get(key)
        
    def add_edge(self, from_key, to_key, weight=0):
        """Add an edge between existing vertices
        
        Vertices are expected to exist, or else a VertexError will
        be raise
        """
        
        if from_key not in self.vertices:
            raise VertexError(f"'From' Vertex key {from_key!r} not found!")
            
        if to_key not in self.vertices:
            raise VertexError(f"'To' Vertex key {to_key!r} not found!")
            
        # lookup the 'from' Vertex object and add a new neighbor to it
        self.vertices[from_key].add_neighbor(self.vertices[to_key], weight)

In [211]:
def build_graph(word_file, count=None, random_select=False):
    d = {}
    g = Graph()
    current = 0
    
    # Pass in a filename, or an already open fileobject
    if isinstance(word_file, str):
        # 'word_file' is a filename, a string
        wfile = open(word_file, 'r')
    else:
        # We should tighten up this conditional check, but it works for now
        # The idea here is word_file can be passed in as an already open file object
        wfile = word_file
        
    # Create the buckets of words that differ by one letter
    for line in wfile:
        # randomly pick a word to avoid loosely connects graph
        # 25% chance of picking a word, or 75% chance of skipping a word    
        if random_select and random() >= 0.05:
            continue
            
        # If required, and count is not None, constrain our number of words used.
        if count and current > count:
            break
            
        word = line.strip()  # stripping '\n' newline character
        for i, _ in enumerate(word):
            bucket = word[:i] + "_" + word[i+1:]
            if bucket in d:
                d[bucket].append(word)
            else:
                d[bucket] = [word]

        current += 1
            
    
    total_connections = 0
        
    # add vertices and edges for words in the same bucket
    for bucket in d:  # iterate over all the keys in dictionary d
        for word1 in d[bucket]:
            for word2 in d[bucket]:
                if word1 != word2:
                    g.add_vertex(word1)
                    g.add_vertex(word2)
                    g.add_edge(word1, word2)
                    total_connections += 1
    
    print(f"Total edges in this graph: {total_connections}")
    return g

In [212]:
def bfs(g, start):
    start.set_distance(0)
    start.set_pred(None)
    vert_queue = Queue()
    vert_queue.enqueue(start)
    
    while vert_queue.size() > 0:
        current_vert = vert_queue.dequeue()
        
        for nbr in current_vert.get_connections():
            if nbr.get_color() == 'white':
                nbr.set_color('gray')
                nbr.set_distance(current_vert.get_distance() + 1)
                nbr.set_pred(current_vert)
                vert_queue.enqueue(nbr)
                
        current_vert.set_color('black')

In [213]:
def ShortPath(g, s, t):
    start = s
    target = t
    curr = target 
    result = bfs(g, start)  
    
    path = [curr] # store the path in a reversed direction: target -> source
      
    while ( curr.get_pred() != start):
        curr = curr.get_pred() # get the precessor vertex
        path.append(curr) # and add in the path
    path.append(source)
                 
    #print
    for ind in range(len(path)-1, 0, -1):
        print(path[ind], '->')
    print(path[0])

## Testing

Test out your solution to show it works as advertised. Use textutal output, or, if you can, perhaps using a program like `graphviz`.

In [214]:
def main():
    words_string = """
foul
foil
fail
fall
pole
pall
poll
pool
cool
fool
pope
pale
sale
sage
page
"""
    words_file = StringIO(words_string)
    word_graph = build_graph(words_file)
    
    return word_graph

In [215]:
wg = main()

Total edges in this graph: 38


In [216]:
len(wg.vertices)

15

In [217]:
ShortPath(wg, wg.get_vertex('fool'),wg.get_vertex('sage'))
# print the shortest path from source = 'fool' to target = 'sage'

fool ->
<Vertex(pool): connected to: ['poll', 'cool', 'fool'] ->
<Vertex(poll): connected to: ['pole', 'pall', 'pool'] ->
<Vertex(pole): connected to: ['pale', 'pope', 'poll'] ->
<Vertex(pale): connected to: ['pole', 'pall', 'sale', 'page'] ->
<Vertex(sale): connected to: ['pale', 'sage'] ->
<Vertex(sage): connected to: ['sale', 'page']
